In [29]:
import requests

In [70]:
standings_url ="https://fbref.com/en/comps/9/Premier-League-Stats"

In [71]:
data = requests.get(standings_url)

In [72]:
from bs4 import BeautifulSoup

In [73]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]


In [75]:
links = standings_table.find_all('a')

In [76]:
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [77]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [78]:
data = requests.get(team_urls[0])

In [79]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [80]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [81]:
data = requests.get(f"https://fbref.com{links[0]}")

In [101]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [102]:
shooting.head()

For Liverpool                                                             \
           Date   Time            Comp        Round  Day Venue Result   GF   
0    2023-08-13  16:30  Premier League  Matchweek 1  Sun  Away      D  1.0   
1    2023-08-19  15:00  Premier League  Matchweek 2  Sat  Home      W  3.0   
2    2023-08-27  16:30  Premier League  Matchweek 3  Sun  Away      W  2.0   
3    2023-09-03  14:00  Premier League  Matchweek 4  Sun  Home      W  3.0   
4    2023-09-16  12:30  Premier League  Matchweek 5  Sat  Away      W  3.0   

                       ... Standard               Expected                    \
    GA       Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0  1.0        Chelsea  ...     17.8  0.0  0     0      1.3  1.3    0.10 -0.3   
1  1.0    Bournemouth  ...     16.8  1.0  0     1      3.0  2.1    0.09  0.0   
2  1.0  Newcastle Utd  ...     17.2  1.0  0     0      0.9  0.9    0.10  1.1   
3  0.0    Aston Villa  ...     14.7  0.0  0     0      2.5  2.5    0.15 -0.5   
4  1.0         Wolves  ...     15.8  0.0  0     0      2.5  2.5    0.16 -0.5   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0    -0.3        Match Report  
1     0.9        Match Report  
2     1.1        Match Report  
3    -0.5        Match Report  
4    -0.5        Match Report  

[5 rows x 26 columns]

In [103]:
shooting.columns = shooting.columns.droplevel()

In [85]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")


In [98]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,4-3-3,Anthony Taylor,Match Report,NaN,13,1,17.8,0.0,0,0
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,4-3-3,Thomas Bramall,Match Report,NaN,25,9,16.8,1.0,0,1
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,4-3-3,John Brooks,Match Report,NaN,9,4,17.2,1.0,0,0
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,4-3-3,Simon Hooper,Match Report,NaN,17,4,14.7,0.0,0,0
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,4-3-3,Michael Oliver,Match Report,NaN,16,5,15.8,0.0,0,0


In [104]:
years = list(range(2024, 2023, -1))
all_matches = []

In [105]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [106]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [114]:
len(all_matches)

20

In [113]:
match_df = pd.concat(all_matches)

In [112]:
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,Match Report,NaN,13.0,1.0,17.8,0.0,0,0,2024,Liverpool
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,Match Report,NaN,25.0,9.0,16.8,1.0,0,1,2024,Liverpool
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,Match Report,NaN,9.0,4.0,17.2,1.0,0,0,2024,Liverpool
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,Match Report,NaN,17.0,4.0,14.7,0.0,0,0,2024,Liverpool
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,Match Report,NaN,16.0,5.0,15.8,0.0,0,0,2024,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,2023-12-06,19:30,Premier League,Matchweek 15,Wed,Home,L,0,2,Liverpool,...,Match Report,NaN,6.0,1.0,14.6,1.0,0,0,2024,Sheffield United
16,2023-12-09,15:00,Premier League,Matchweek 16,Sat,Home,W,1,0,Brentford,...,Match Report,NaN,9.0,4.0,21.7,0.0,0,0,2024,Sheffield United
17,2023-12-16,15:00,Premier League,Matchweek 17,Sat,Away,L,0,2,Chelsea,...,Match Report,NaN,6.0,1.0,19.7,1.0,0,0,2024,Sheffield United
18,2023-12-22,20:00,Premier League,Matchweek 18,Fri,Away,D,1,1,Aston Villa,...,Match Report,NaN,5.0,2.0,19.7,0.0,0,0,2024,Sheffield United


In [115]:
match_df.to_csv("premiere_league.csv")